# Chapter 1: Mechanisms of Action (MoA) Prediction using DNN model

## Background

The Connectivity Map, a project within the Broad Institute of MIT and Harvard, together with the Laboratory for Innovation Science at Harvard (LISH), presents a challenge [at kaggle](https://www.kaggle.com/c/lish-moa/overview) with the goal of advancing drug development through improvements to MoA prediction algorithms.

We will be predicting multiple targets of the Mechanism of Action (MoA) response(s) of different samples (sig_id), given various inputs such as gene expression data and cell viability data.

Two notes:

- the training data has an additional (optional) set of MoA labels that are not included in the test data and not used for scoring.
- the re-run dataset has approximately 4x the number of examples seen in the Public test.

## Data

- train_features.csv - Features for the training set. Features g- signify gene expression data, and c- signify cell viability data. cp_type indicates samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); control perturbations have no MoAs; cp_time and cp_dose indicate treatment duration (24, 48, 72 hours) and dose (high or low).
- train_targets_scored.csv - The binary MoA targets that are scored.
- train_targets_nonscored.csv - Additional (optional) binary MoA responses for the training data. These are not predicted nor scored.
- test_features.csv - Features for the test data. You must predict the probability of each scored MoA for each row in the test data.
- sample_submission.csv - A submission file in the correct format.

## Import Libraries


In [2]:
import numpy as np 
import pandas as pd 
import os

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
 
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M

In [3]:
!pwd

'pwd' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


## Prepare Dataset

### Copy datasets from course project directory if not exist

In [19]:
from distutils.dir_util import copy_tree
import os.path
from os import path
if not path.exists('train_features.csv'):
    copy_tree("/fs/ess/PAS1791/course_code/dnn_example", ".")

### Load datasets, note we only use the first 1000 lines for the example

In [20]:
test_df = pd.read_csv('./test_features.csv')
train_df = pd.read_csv('./train_features.csv')
train_target_df = pd.read_csv('./train_targets_scored.csv')

target_cols = train_target_df.columns[1:]
N_TARGETS = len(target_cols)

In [21]:
test_df = test_df.iloc[0:1000:,]
train_df = train_df.iloc[0:1000,:]


In [22]:
test_df.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


In [23]:
train_df.head()


,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


### Basic Setup and Helpers


In [25]:
SEED = 1234
EPOCHS = 4
BATCH_SIZE = 16
FOLDS = 3
REPEATS = 2
LR = 0.05
N_TARGETS = len(target_cols)

In [26]:
def seed_everything(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

In [27]:
def multi_log_loss(y_true, y_pred):
    losses = []
    for col in y_true.columns:
        losses.append(log_loss(y_true.loc[:, col], y_pred.loc[:, col]))
    return np.mean(losses)

Encode Categoricals to Binary¶


In [28]:
def preprocess_df(data):
    data['cp_type'] = (data['cp_type'] == 'trt_cp').astype(int)
    data['cp_dose'] = (data['cp_dose'] == 'D2').astype(int)
    return data

In [29]:
x_train = preprocess_df(train_df.drop(columns="sig_id"))
x_test =preprocess_df(test_df.drop(columns="sig_id"))
y_train = train_target_df.drop(columns="sig_id")
N_FEATURES = x_train.shape[1]

Define Model Architecture¶


In [30]:
def create_model():
    model = tf.keras.Sequential([
    tf.keras.layers.Input(N_FEATURES),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    #tf.keras.layers.Dropout(0.4),
    #tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),  
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(N_TARGETS, activation="sigmoid"))
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = LR), loss='binary_crossentropy', metrics=["accuracy"])
    return model

Main CV and Model Training Function¶


In [31]:
def build_train(resume_models = None, repeat_number = 0, folds = 3, skip_folds = 0):
    
    models = []
    oof_preds = y_train.copy()
    

    kfold = KFold(folds, shuffle = True)
    for fold, (train_ind, val_ind) in enumerate(kfold.split(x_train)):
        print('\n')
        print('-'*50)
        print(f'Training fold {fold + 1}')
        
        cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'binary_crossentropy', factor = 0.4, patience = 2, verbose = 1, min_delta = 0.0001, mode = 'auto')
        checkpoint_path = f'repeat:{repeat_number}_Fold:{fold}.hdf5'
        cb_checkpt = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True, save_weights_only = True, mode = 'min')

        model = create_model()
        model.fit(x_train.values[train_ind],
              y_train.values[train_ind],
              validation_data=(x_train.values[val_ind], y_train.values[val_ind]),
              callbacks = [cb_lr_schedule, cb_checkpt],
              epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=2
             )
        model.load_weights(checkpoint_path)
        oof_preds.loc[val_ind, :] = model.predict(x_train.values[val_ind])
        models.append(model)

    return models, oof_preds

In [32]:
models = []
oof_preds = []
# seed everything
seed_everything(SEED)
for i in range(REPEATS):
    m, oof = build_train(repeat_number = i, folds=FOLDS)
    models = models + m
    oof_preds.append(oof)



--------------------------------------------------
Training fold 1
Epoch 1/4
42/42 - 3s - loss: 0.1007 - accuracy: 0.0165 - val_loss: 0.0375 - val_accuracy: 0.0150 - lr: 0.0500
Epoch 2/4
42/42 - 3s - loss: 0.0214 - accuracy: 0.0345 - val_loss: 0.0291 - val_accuracy: 0.0090 - lr: 0.0500
Epoch 3/4
42/42 - 3s - loss: 0.0204 - accuracy: 0.0315 - val_loss: 0.0240 - val_accuracy: 0.0449 - lr: 0.0500
Epoch 4/4
42/42 - 3s - loss: 0.0198 - accuracy: 0.0420 - val_loss: 0.0231 - val_accuracy: 0.0299 - lr: 0.0500


OSError: Unable to open file (unable to open file: name = 'repeat:0_Fold:0.hdf5', errno = 22, error message = 'Invalid argument', flags = 0, o_flags = 0)

In [156]:
mean_oof_preds = y_train.copy()
mean_oof_preds.loc[:, target_cols] = 0
for i, p in enumerate(oof_preds):
    print(f"Repeat {i + 1} OOF Log Loss: {multi_log_loss(y_train, p)}")
    mean_oof_preds.loc[:, target_cols] += p[target_cols]

mean_oof_preds.loc[:, target_cols] /= len(oof_preds)
print(f"Mean OOF Log Loss: {multi_log_loss(y_train, mean_oof_preds)}")
#mean_oof_preds.loc[x_train['cp_type'] == 0, target_cols] = 0
#print(f"Mean OOF Log Loss (ctl adjusted): {multi_log_loss(y_train, mean_oof_preds)}")

Repeat 1 OOF Log Loss: 0.0009779393583764593
Repeat 2 OOF Log Loss: 0.0009877390165540364
Mean OOF Log Loss: 0.0009210642085450577


Make Test Predictions and Save Submission¶


Make Test Predictions and Save Submission¶


In [157]:
test_preds = sub.copy()
test_preds[target_cols] = 0
for model in models:
    test_preds.loc[:,target_cols] += model.predict(x_test)
test_preds.loc[:,target_cols] /= len(models)
test_preds.loc[x_test['cp_type'] == 0, target_cols] = 0
test_preds.to_csv('submission.csv', index=False)